In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Find only CSV files and filter these which were recorded since 2017-01-04

In [2]:
def Webscrap_links_with_CSV():
    ## CSV files are grouped in table on WWW site
    ## https://cycling.data.tfl.gov.uk/ in folder usage-stats

    url_name = 'https://cycling.data.tfl.gov.uk/usage-stats/'
    html_file = requests.get(url_name).text
    soup = BeautifulSoup(html_file, 'lxml')

    return soup.findAll('a')


website_links = Webscrap_links_with_CSV()

#### First method failed (empty list means dynamic website - JS scripts were used which give full HTML code only on the server side). We have to look for JSON files, XHR, GET method or some API - only after is a chance of getting info about weather in any day we want.

In [3]:
url_name ='https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/'
html_file = requests.get(url_name).text
html_file

'<?xml version="1.0" encoding="UTF-8"?>\n<ListBucketResult xmlns="http://s3.amazonaws.com/doc/2006-03-01/"><Name>cycling.data.tfl.gov.uk</Name><Prefix></Prefix><Marker></Marker><MaxKeys>1000</MaxKeys><IsTruncated>false</IsTruncated><Contents><Key>CycleCounters/</Key><LastModified>2018-08-29T11:05:25.000Z</LastModified><ETag>&quot;d41d8cd98f00b204e9800998ecf8427e&quot;</ETag><Size>0</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key>CycleCounters/Blackfriars/</Key><LastModified>2018-08-29T11:17:55.000Z</LastModified><ETag>&quot;d41d8cd98f00b204e9800998ecf8427e&quot;</ETag><Size>0</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key>CycleCounters/Blackfriars/July/</Key><LastModified>2018-08-29T11:17:55.000Z</LastModified><ETag>&quot;d41d8cd98f00b204e9800998ecf8427e&quot;</ETag><Size>0</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key>CycleCounters/Blackfriars/July/Friday, Jul 13 2018.xls</Key><LastModified>2018-08-29T11:17:55.000Z</Las

#### Let's inspect positions 2-6 which include 'usage-stats' phrase

In [4]:
for i in range(2,7):
    print(html_file.split('usage-stats')[i], '\n')

/01aJourneyDataExtract10Jan16-23Jan16.csv</Key><LastModified>2016-08-24T14:46:34.000Z</LastModified><ETag>&quot;884af1def5ce69e4d162a814c8f74ca7&quot;</ETag><Size>34451153</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key> 

/01b Journey Data Extract 24Jan16-06Feb16.csv</Key><LastModified>2016-08-23T09:16:33.000Z</LastModified><ETag>&quot;7e7e6cccb0dca066844a72dab524b527&quot;</ETag><Size>35486081</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key> 

/01bJourneyDataExtract24Jan16-06Feb16.csv</Key><LastModified>2016-08-24T14:46:42.000Z</LastModified><ETag>&quot;7e7e6cccb0dca066844a72dab524b527&quot;</ETag><Size>35486081</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key> 

/02aJourneyDataExtract07Fe16-20Feb2016.csv</Key><LastModified>2016-03-11T11:38:10.000Z</LastModified><ETag>&quot;1ada36f04a9b1ce64245f6fc775f73ce&quot;</ETag><Size>33287674</Size><StorageClass>STANDARD</StorageClass></Contents><Contents><Key> 

/02bJourneyDataExtr

In [5]:
import re

def filter_files_name(text):
    _ = []
    str_split = (re.findall(text, html_file)[0]).split('</Key>')
    for i in range(0, len(str_split)):
        if len(str_split)!=0:
            if re.findall(r"^usage-stats", str_split[i]):
                _.append(str_split[i])
            else:
                _.append(re.sub('<LastModified.+<Key>', '', str_split[i]))
    return _
     

CSV_list = filter_files_name(r"usage-stats\/\d.+\d{2}\Wcsv</Key>")
CSV_list


['usage-stats/01aJourneyDataExtract10Jan16-23Jan16.csv',
 'usage-stats/01b Journey Data Extract 24Jan16-06Feb16.csv',
 'usage-stats/01bJourneyDataExtract24Jan16-06Feb16.csv',
 'usage-stats/02aJourneyDataExtract07Fe16-20Feb2016.csv',
 'usage-stats/02bJourneyDataExtract21Feb16-05Mar2016.csv',
 'usage-stats/03JourneyDataExtract06Mar2016-31Mar2016.csv',
 'usage-stats/04JourneyDataExtract01Apr2016-30Apr2016.csv',
 'usage-stats/05JourneyDataExtract01May2016-17May2016.csv',
 'usage-stats/06JourneyDataExtract18May2016-24May2016.csv',
 'usage-stats/07JourneyDataExtract25May2016-31May2016.csv',
 'usage-stats/08JourneyDataExtract01Jun2016-07Jun2016.csv',
 'usage-stats/09JourneyDataExtract08Jun2016-14Jun2016.csv',
 'usage-stats/100JourneyDataExtract07Mar2018-13Mar2018.csv',
 'usage-stats/101JourneyDataExtract14Mar2018-20Mar2018.csv',
 'usage-stats/102JourneyDataExtract21Mar2018-27Mar2018.csv',
 'usage-stats/103JourneyDataExtract28Mar2018-03Apr2018.csv',
 'usage-stats/104JourneyDataExtract04Apr2018

#### Now we are looking for years range in format 2017-2022 or 17-22

In [6]:
files_url = []

for i in range(0, len(CSV_list)):
    regex1 = re.findall(r".+1[7-9]\.csv", CSV_list[i])
    regex2 = re.findall(r".+2[0-2]\.csv", CSV_list[i])
    if regex1:
        files_url.append(regex1[0])
    if regex2:
        files_url.append(regex2[0])

#### First 3 strings from the list as an example

In [7]:
files_url[:3]

['usage-stats/100JourneyDataExtract07Mar2018-13Mar2018.csv',
 'usage-stats/101JourneyDataExtract14Mar2018-20Mar2018.csv',
 'usage-stats/102JourneyDataExtract21Mar2018-27Mar2018.csv']

In [8]:
## Concatenate strings to make full URL path to files
for i in range(0, len(files_url)):
    files_url[i] = f'{url_name}{files_url[i]}'

In [9]:
files_url[:3]

['https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/usage-stats/100JourneyDataExtract07Mar2018-13Mar2018.csv',
 'https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/usage-stats/101JourneyDataExtract14Mar2018-20Mar2018.csv',
 'https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/usage-stats/102JourneyDataExtract21Mar2018-27Mar2018.csv']

#### Some of the URL strings contain whitescapes inside CSV file names and we must replace every ' ' with '%20'

In [11]:
files_url = [files_url[i].replace(" ", "%20") for i in range(0, len(files_url))]

files_url[:3]

['https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/usage-stats/100JourneyDataExtract07Mar2018-13Mar2018.csv',
 'https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/usage-stats/101JourneyDataExtract14Mar2018-20Mar2018.csv',
 'https://s3-eu-west-1.amazonaws.com/cycling.data.tfl.gov.uk/usage-stats/102JourneyDataExtract21Mar2018-27Mar2018.csv']

### Now we are ready to save results into txt file and use it later

In [11]:
import os

os.chdir("CSV_files")

In [12]:
with open("CSV_files_url.txt", 'w') as f:
    for s in files_url:
        f.write(f'{str(s)}\n')